# Improved EDA and Error Analysis for Employee Compensation Prediction

This notebook provides an enhanced exploratory data analysis (EDA) with comprehensive error calculation and model evaluation techniques for employee compensation prediction.

## 1. Enhanced Data Loading and Initial Setup

In [ ]:
# Enhanced imports for comprehensive analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import shapiro, jarque_bera, normaltest, anderson
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tools.eval_measures import rmse
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score, 
    mean_absolute_percentage_error, explained_variance_score
)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Enhanced display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.6f}'.format)
np.random.seed(42)

In [ ]:
# Load data with enhanced error handling
try:
    df = pd.read_csv('employee_compensation 3.csv')
    print(f"✅ Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns")
except FileNotFoundError:
    print("❌ File not found. Please ensure 'employee_compensation 3.csv' is in the current directory.")
    # Create sample data for demonstration
    np.random.seed(42)
    n_samples = 5000
    df = pd.DataFrame({
        'Type_of_Year': np.random.choice(['Financial', 'Calendar'], n_samples),
        'Year': np.random.choice([2013, 2014, 2015], n_samples),
        'Emp_ID': range(1, n_samples + 1),
        'Income': np.random.normal(75000, 25000, n_samples),
        'Overtime': np.random.exponential(5000, n_samples),
        'Other_Income': np.random.exponential(3000, n_samples),
        'Total_Income': lambda x: x['Income'] + x['Overtime'] + x['Other_Income'],
        'Retirement': lambda x: x['Total_Income'] * 0.15 + np.random.normal(0, 1000, n_samples),
        'Health_Insurance': np.random.normal(8000, 2000, n_samples),
        'Other_Benefits': np.random.normal(4000, 1500, n_samples),
    })
    df['Total_Benefits'] = df['Retirement'] + df['Health_Insurance'] + df['Other_Benefits']
    df['Total_Reimbursement'] = df['Total_Income'] + df['Total_Benefits']
    print(f"✅ Sample data created: {df.shape[0]} rows, {df.shape[1]} columns")

# Display basic information
print("\n📊 Dataset Overview:")
print(df.info())
print("\n📈 First 5 rows:")
df.head()

## 2. Comprehensive Data Quality Assessment

In [ ]:
def comprehensive_data_quality_report(df):
    """
    Generate a comprehensive data quality report
    """
    print("🔍 COMPREHENSIVE DATA QUALITY REPORT")
    print("=" * 50)
    
    # Basic statistics
    print(f"📊 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Missing values analysis
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing_data,
        'Missing_Percentage': missing_percent
    }).sort_values('Missing_Count', ascending=False)
    
    print("\n🚫 Missing Values Analysis:")
    if missing_df['Missing_Count'].sum() == 0:
        print("✅ No missing values found!")
    else:
        print(missing_df[missing_df['Missing_Count'] > 0])
    
    # Data types analysis
    print("\n📋 Data Types:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  {dtype}: {count} columns")
    
    # Duplicate analysis
    duplicates = df.duplicated().sum()
    print(f"\n🔄 Duplicate Rows: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
    
    # Numerical columns analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(f"\n🔢 Numerical Columns: {len(numeric_cols)}")
    
    # Categorical columns analysis
    categorical_cols = df.select_dtypes(include=['object']).columns
    print(f"📝 Categorical Columns: {len(categorical_cols)}")
    
    if len(categorical_cols) > 0:
        print("\n📊 Categorical Variables Cardinality:")
        for col in categorical_cols:
            unique_count = df[col].nunique()
            print(f"  {col}: {unique_count} unique values")
    
    return missing_df, numeric_cols, categorical_cols

missing_analysis, numeric_columns, categorical_columns = comprehensive_data_quality_report(df)

## 3. Advanced Statistical Analysis

In [ ]:
def advanced_statistical_summary(df, numeric_cols):
    """
    Generate advanced statistical summary with distribution analysis
    """
    print("📊 ADVANCED STATISTICAL SUMMARY")
    print("=" * 40)
    
    stats_summary = []
    
    for col in numeric_cols:
        data = df[col].dropna()
        
        # Basic statistics
        basic_stats = {
            'Column': col,
            'Count': len(data),
            'Mean': data.mean(),
            'Median': data.median(),
            'Mode': data.mode().iloc[0] if len(data.mode()) > 0 else np.nan,
            'Std': data.std(),
            'Variance': data.var(),
            'Min': data.min(),
            'Max': data.max(),
            'Range': data.max() - data.min(),
            'Q1': data.quantile(0.25),
            'Q3': data.quantile(0.75),
            'IQR': data.quantile(0.75) - data.quantile(0.25),
            'Skewness': stats.skew(data),
            'Kurtosis': stats.kurtosis(data),
            'CV': data.std() / data.mean() if data.mean() != 0 else np.nan
        }
        
        # Normality tests
        try:
            shapiro_stat, shapiro_p = shapiro(data.sample(min(5000, len(data))))
            basic_stats['Shapiro_p'] = shapiro_p
            basic_stats['Is_Normal_Shapiro'] = shapiro_p > 0.05
        except:
            basic_stats['Shapiro_p'] = np.nan
            basic_stats['Is_Normal_Shapiro'] = False
        
        try:
            jb_stat, jb_p = jarque_bera(data)
            basic_stats['JB_p'] = jb_p
            basic_stats['Is_Normal_JB'] = jb_p > 0.05
        except:
            basic_stats['JB_p'] = np.nan
            basic_stats['Is_Normal_JB'] = False
        
        # Outlier detection using IQR method
        Q1, Q3 = data.quantile(0.25), data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        basic_stats['Outliers_Count'] = len(outliers)
        basic_stats['Outliers_Percentage'] = len(outliers) / len(data) * 100
        
        stats_summary.append(basic_stats)
    
    stats_df = pd.DataFrame(stats_summary)
    return stats_df

# Generate advanced statistical summary
advanced_stats = advanced_statistical_summary(df, numeric_columns)
print("\n📊 Advanced Statistical Summary:")
display(advanced_stats.round(4))

## 4. Enhanced Visualization Suite

In [ ]:
def create_comprehensive_visualization_suite(df, numeric_cols, target_col='Total_Reimbursement'):
    """
    Create comprehensive visualization suite for EDA
    """
    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    
    # 1. Distribution Analysis
    n_cols = min(4, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(numeric_cols[:len(axes)]):
        # Histogram with KDE
        sns.histplot(data=df, x=col, kde=True, ax=axes[i], alpha=0.7)
        axes[i].axvline(df[col].mean(), color='red', linestyle='--', alpha=0.8, label=f'Mean: {df[col].mean():.2f}')
        axes[i].axvline(df[col].median(), color='green', linestyle='--', alpha=0.8, label=f'Median: {df[col].median():.2f}')
        axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    # Hide empty subplots
    for i in range(len(numeric_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('📊 Distribution Analysis of Numerical Variables', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    # 2. Correlation Heatmap with Enhanced Features
    plt.figure(figsize=(14, 10))
    correlation_matrix = df[numeric_cols].corr()
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    # Generate heatmap
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                cmap='RdYlBu_r', 
                center=0,
                square=True,
                fmt='.3f',
                cbar_kws={'shrink': 0.8})
    
    plt.title('🔗 Enhanced Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # 3. Box Plots for Outlier Detection
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(numeric_cols[:len(axes)]):
        sns.boxplot(data=df, y=col, ax=axes[i])
        axes[i].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
        axes[i].grid(True, alpha=0.3)
        
        # Add statistical annotations
        q1, q3 = df[col].quantile([0.25, 0.75])
        iqr = q3 - q1
        outliers = df[(df[col] < q1 - 1.5*iqr) | (df[col] > q3 + 1.5*iqr)][col]
        axes[i].text(0.02, 0.98, f'Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)', 
                    transform=axes[i].transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Hide empty subplots
    for i in range(len(numeric_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('📦 Outlier Detection via Box Plots', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    # 4. Pairwise Relationships (focusing on target variable)
    if target_col in numeric_cols:
        other_cols = [col for col in numeric_cols if col != target_col][:6]  # Limit to 6 for readability
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, col in enumerate(other_cols[:6]):
            sns.scatterplot(data=df, x=col, y=target_col, ax=axes[i], alpha=0.6)
            
            # Add regression line
            sns.regplot(data=df, x=col, y=target_col, ax=axes[i], 
                       scatter=False, color='red', line_kws={'linewidth': 2})
            
            # Calculate and display correlation
            corr = df[col].corr(df[target_col])
            axes[i].set_title(f'{col} vs {target_col}\nCorrelation: {corr:.3f}', 
                             fontsize=11, fontweight='bold')
            axes[i].grid(True, alpha=0.3)
        
        # Hide empty subplots
        for i in range(len(other_cols), 6):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.suptitle(f'🎯 Relationships with Target Variable: {target_col}', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.show()

# Create comprehensive visualizations
create_comprehensive_visualization_suite(df, numeric_columns)

## 5. Advanced Model Building with Cross-Validation

In [ ]:
def prepare_data_for_modeling(df, target_col='Total_Reimbursement'):
    """
    Prepare data for modeling with proper preprocessing
    """
    # Select features (excluding target and ID columns)
    feature_cols = [col for col in df.columns if col not in [target_col, 'Emp_ID']]
    
    # Handle categorical variables
    df_processed = df.copy()
    
    # One-hot encode categorical variables
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        df_processed = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True)
    
    # Update feature columns after encoding
    feature_cols = [col for col in df_processed.columns if col not in [target_col, 'Emp_ID']]
    
    X = df_processed[feature_cols]
    y = df_processed[target_col]
    
    return X, y, feature_cols

# Prepare data
X, y, feature_names = prepare_data_for_modeling(df)
print(f"✅ Data prepared: {X.shape[0]} samples, {X.shape[1]} features")
print(f"📊 Features: {feature_names}")

In [ ]:
def comprehensive_model_evaluation(X, y, test_size=0.2, random_state=42):
    """
    Comprehensive model evaluation with multiple algorithms and metrics
    """
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    # Define models to evaluate
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=1.0),
        'Elastic Net': ElasticNet(alpha=1.0, l1_ratio=0.5),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=random_state),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=random_state)
    }
    
    # Initialize results storage
    results = []
    predictions = {}
    
    # Cross-validation setup
    cv = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    print("🔄 Training and evaluating models...")
    print("=" * 50)
    
    for name, model in models.items():
        print(f"\n🤖 Training {name}...")
        
        # Cross-validation scores
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
        cv_rmse = -cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
        cv_mae = -cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')
        
        # Fit model and make predictions
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Store predictions for later analysis
        predictions[name] = {
            'train_pred': y_pred_train,
            'test_pred': y_pred_test,
            'train_actual': y_train,
            'test_actual': y_test
        }
        
        # Calculate comprehensive metrics
        train_metrics = calculate_regression_metrics(y_train, y_pred_train)
        test_metrics = calculate_regression_metrics(y_test, y_pred_test)
        
        # Store results
        result = {
            'Model': name,
            'CV_R2_Mean': cv_scores.mean(),
            'CV_R2_Std': cv_scores.std(),
            'CV_RMSE_Mean': cv_rmse.mean(),
            'CV_RMSE_Std': cv_rmse.std(),
            'CV_MAE_Mean': cv_mae.mean(),
            'CV_MAE_Std': cv_mae.std(),
            'Train_R2': train_metrics['R2'],
            'Test_R2': test_metrics['R2'],
            'Train_RMSE': train_metrics['RMSE'],
            'Test_RMSE': test_metrics['RMSE'],
            'Train_MAE': train_metrics['MAE'],
            'Test_MAE': test_metrics['MAE'],
            'Train_MAPE': train_metrics['MAPE'],
            'Test_MAPE': test_metrics['MAPE'],
            'Overfitting_Score': train_metrics['R2'] - test_metrics['R2']
        }
        
        results.append(result)
        
        # Print summary
        print(f"  ✅ CV R²: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
        print(f"  📊 Test R²: {test_metrics['R2']:.4f}")
        print(f"  📉 Test RMSE: {test_metrics['RMSE']:.2f}")
        print(f"  📈 Overfitting: {result['Overfitting_Score']:.4f}")
    
    results_df = pd.DataFrame(results)
    return results_df, predictions, (X_train, X_test, y_train, y_test)

def calculate_regression_metrics(y_true, y_pred):
    """
    Calculate comprehensive regression metrics
    """
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,
        'Explained_Variance': explained_variance_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred)
    }

# Run comprehensive model evaluation
model_results, model_predictions, data_splits = comprehensive_model_evaluation(X, y)

print("\n📊 COMPREHENSIVE MODEL COMPARISON")
print("=" * 60)
display(model_results.round(4))

## 6. Advanced Error Analysis and Diagnostics

In [ ]:
def advanced_error_analysis(model_results, model_predictions, data_splits):
    """
    Perform advanced error analysis and diagnostics
    """
    X_train, X_test, y_train, y_test = data_splits
    
    # 1. Model Performance Comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # R² Comparison
    models = model_results['Model']
    train_r2 = model_results['Train_R2']
    test_r2 = model_results['Test_R2']
    
    x = np.arange(len(models))
    width = 0.35
    
    axes[0,0].bar(x - width/2, train_r2, width, label='Train R²', alpha=0.8)
    axes[0,0].bar(x + width/2, test_r2, width, label='Test R²', alpha=0.8)
    axes[0,0].set_xlabel('Models')
    axes[0,0].set_ylabel('R² Score')
    axes[0,0].set_title('R² Score Comparison: Train vs Test')
    axes[0,0].set_xticks(x)
    axes[0,0].set_xticklabels(models, rotation=45, ha='right')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # RMSE Comparison
    train_rmse = model_results['Train_RMSE']
    test_rmse = model_results['Test_RMSE']
    
    axes[0,1].bar(x - width/2, train_rmse, width, label='Train RMSE', alpha=0.8)
    axes[0,1].bar(x + width/2, test_rmse, width, label='Test RMSE', alpha=0.8)
    axes[0,1].set_xlabel('Models')
    axes[0,1].set_ylabel('RMSE')
    axes[0,1].set_title('RMSE Comparison: Train vs Test')
    axes[0,1].set_xticks(x)
    axes[0,1].set_xticklabels(models, rotation=45, ha='right')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Cross-Validation Performance
    cv_r2_mean = model_results['CV_R2_Mean']
    cv_r2_std = model_results['CV_R2_Std']
    
    axes[1,0].bar(x, cv_r2_mean, yerr=cv_r2_std, capsize=5, alpha=0.8)
    axes[1,0].set_xlabel('Models')
    axes[1,0].set_ylabel('CV R² Score')
    axes[1,0].set_title('Cross-Validation R² Score (Mean ± Std)')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(models, rotation=45, ha='right')
    axes[1,0].grid(True, alpha=0.3)
    
    # Overfitting Analysis
    overfitting_scores = model_results['Overfitting_Score']
    colors = ['red' if score > 0.1 else 'orange' if score > 0.05 else 'green' for score in overfitting_scores]
    
    axes[1,1].bar(x, overfitting_scores, color=colors, alpha=0.8)
    axes[1,1].axhline(y=0.05, color='orange', linestyle='--', alpha=0.7, label='Moderate Overfitting')
    axes[1,1].axhline(y=0.1, color='red', linestyle='--', alpha=0.7, label='High Overfitting')
    axes[1,1].set_xlabel('Models')
    axes[1,1].set_ylabel('Overfitting Score (Train R² - Test R²)')
    axes[1,1].set_title('Overfitting Analysis')
    axes[1,1].set_xticks(x)
    axes[1,1].set_xticklabels(models, rotation=45, ha='right')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('🔍 Advanced Model Performance Analysis', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    # 2. Residual Analysis for Best Model
    best_model_idx = model_results['Test_R2'].idxmax()
    best_model_name = model_results.loc[best_model_idx, 'Model']
    
    print(f"\n🏆 Best Model: {best_model_name}")
    print(f"📊 Test R²: {model_results.loc[best_model_idx, 'Test_R2']:.4f}")
    print(f"📉 Test RMSE: {model_results.loc[best_model_idx, 'Test_RMSE']:.2f}")
    
    # Get predictions for best model
    best_predictions = model_predictions[best_model_name]
    y_test_pred = best_predictions['test_pred']
    y_test_actual = best_predictions['test_actual']
    
    # Calculate residuals
    residuals = y_test_actual - y_test_pred
    
    # Residual Analysis Plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Actual vs Predicted
    axes[0,0].scatter(y_test_actual, y_test_pred, alpha=0.6)
    axes[0,0].plot([y_test_actual.min(), y_test_actual.max()], 
                   [y_test_actual.min(), y_test_actual.max()], 'r--', lw=2)
    axes[0,0].set_xlabel('Actual Values')
    axes[0,0].set_ylabel('Predicted Values')
    axes[0,0].set_title('Actual vs Predicted Values')
    axes[0,0].grid(True, alpha=0.3)
    
    # Add R² annotation
    r2_test = r2_score(y_test_actual, y_test_pred)
    axes[0,0].text(0.05, 0.95, f'R² = {r2_test:.4f}', transform=axes[0,0].transAxes,
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 2. Residuals vs Predicted
    axes[0,1].scatter(y_test_pred, residuals, alpha=0.6)
    axes[0,1].axhline(y=0, color='r', linestyle='--')
    axes[0,1].set_xlabel('Predicted Values')
    axes[0,1].set_ylabel('Residuals')
    axes[0,1].set_title('Residuals vs Predicted Values')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Residual Distribution
    sns.histplot(residuals, kde=True, ax=axes[0,2])
    axes[0,2].axvline(residuals.mean(), color='red', linestyle='--', 
                      label=f'Mean: {residuals.mean():.2f}')
    axes[0,2].set_xlabel('Residuals')
    axes[0,2].set_ylabel('Frequency')
    axes[0,2].set_title('Distribution of Residuals')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Q-Q Plot for Residuals
    stats.probplot(residuals, dist="norm", plot=axes[1,0])
    axes[1,0].set_title('Q-Q Plot of Residuals')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Scale-Location Plot
    standardized_residuals = np.sqrt(np.abs(residuals / residuals.std()))
    axes[1,1].scatter(y_test_pred, standardized_residuals, alpha=0.6)
    axes[1,1].set_xlabel('Predicted Values')
    axes[1,1].set_ylabel('√|Standardized Residuals|')
    axes[1,1].set_title('Scale-Location Plot')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Error Distribution by Prediction Range
    pred_ranges = pd.cut(y_test_pred, bins=5, labels=['Low', 'Low-Med', 'Medium', 'Med-High', 'High'])
    error_by_range = pd.DataFrame({'Range': pred_ranges, 'Absolute_Error': np.abs(residuals)})
    sns.boxplot(data=error_by_range, x='Range', y='Absolute_Error', ax=axes[1,2])
    axes[1,2].set_title('Error Distribution by Prediction Range')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle(f'🔬 Residual Analysis: {best_model_name}', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    return best_model_name, residuals

# Perform advanced error analysis
best_model, residuals = advanced_error_analysis(model_results, model_predictions, data_splits)

## 7. Statistical Tests and Model Diagnostics

In [ ]:
def comprehensive_model_diagnostics(residuals, X_test, y_test, y_pred):
    """
    Perform comprehensive statistical tests and diagnostics
    """
    print("🧪 COMPREHENSIVE MODEL DIAGNOSTICS")
    print("=" * 50)
    
    diagnostics_results = {}
    
    # 1. Normality Tests for Residuals
    print("\n📊 NORMALITY TESTS FOR RESIDUALS:")
    print("-" * 35)
    
    # Shapiro-Wilk Test
    if len(residuals) <= 5000:
        shapiro_stat, shapiro_p = shapiro(residuals)
        print(f"Shapiro-Wilk Test:")
        print(f"  Statistic: {shapiro_stat:.6f}")
        print(f"  p-value: {shapiro_p:.6f}")
        print(f"  Normal: {'✅ Yes' if shapiro_p > 0.05 else '❌ No'}")
        diagnostics_results['shapiro_p'] = shapiro_p
    else:
        print("Shapiro-Wilk Test: Skipped (sample too large)")
    
    # Jarque-Bera Test
    jb_stat, jb_p = jarque_bera(residuals)
    print(f"\nJarque-Bera Test:")
    print(f"  Statistic: {jb_stat:.6f}")
    print(f"  p-value: {jb_p:.6f}")
    print(f"  Normal: {'✅ Yes' if jb_p > 0.05 else '❌ No'}")
    diagnostics_results['jb_p'] = jb_p
    
    # Anderson-Darling Test
    ad_stat, ad_critical, ad_significance = anderson(residuals, dist='norm')
    print(f"\nAnderson-Darling Test:")
    print(f"  Statistic: {ad_stat:.6f}")
    print(f"  Critical Values: {ad_critical}")
    print(f"  Significance Levels: {ad_significance}%")
    normal_ad = ad_stat < ad_critical[2]  # 5% significance level
    print(f"  Normal (5% level): {'✅ Yes' if normal_ad else '❌ No'}")
    diagnostics_results['anderson_normal'] = normal_ad
    
    # 2. Homoscedasticity Tests
    print("\n🎯 HOMOSCEDASTICITY TESTS:")
    print("-" * 30)
    
    # Prepare data for heteroscedasticity tests
    X_with_const = sm.add_constant(X_test)
    
    try:
        # Breusch-Pagan Test
        bp_stat, bp_p, bp_f_stat, bp_f_p = het_breuschpagan(residuals, X_with_const)
        print(f"Breusch-Pagan Test:")
        print(f"  LM Statistic: {bp_stat:.6f}")
        print(f"  p-value: {bp_p:.6f}")
        print(f"  Homoscedastic: {'✅ Yes' if bp_p > 0.05 else '❌ No'}")
        diagnostics_results['bp_p'] = bp_p
        
        # White Test
        white_stat, white_p, white_f_stat, white_f_p = het_white(residuals, X_with_const)
        print(f"\nWhite Test:")
        print(f"  LM Statistic: {white_stat:.6f}")
        print(f"  p-value: {white_p:.6f}")
        print(f"  Homoscedastic: {'✅ Yes' if white_p > 0.05 else '❌ No'}")
        diagnostics_results['white_p'] = white_p
        
    except Exception as e:
        print(f"Heteroscedasticity tests failed: {e}")
    
    # 3. Error Distribution Analysis
    print("\n📈 ERROR DISTRIBUTION ANALYSIS:")
    print("-" * 35)
    
    abs_errors = np.abs(residuals)
    squared_errors = residuals ** 2
    
    print(f"Mean Absolute Error: {abs_errors.mean():.4f}")
    print(f"Median Absolute Error: {np.median(abs_errors):.4f}")
    print(f"90th Percentile Error: {np.percentile(abs_errors, 90):.4f}")
    print(f"95th Percentile Error: {np.percentile(abs_errors, 95):.4f}")
    print(f"99th Percentile Error: {np.percentile(abs_errors, 99):.4f}")
    
    # Error percentiles
    error_percentiles = [50, 75, 90, 95, 99]
    print(f"\nError Percentiles:")
    for p in error_percentiles:
        print(f"  {p}th percentile: {np.percentile(abs_errors, p):.4f}")
    
    # 4. Prediction Intervals
    print("\n🎯 PREDICTION INTERVALS:")
    print("-" * 25)
    
    residual_std = residuals.std()
    
    # Calculate prediction intervals (assuming normal distribution)
    confidence_levels = [0.68, 0.95, 0.99]
    z_scores = [1.0, 1.96, 2.576]
    
    for conf, z in zip(confidence_levels, z_scores):
        interval_width = z * residual_std
        coverage = np.mean(np.abs(residuals) <= interval_width) * 100
        print(f"{conf*100:.0f}% Prediction Interval:")
        print(f"  Width: ±{interval_width:.4f}")
        print(f"  Actual Coverage: {coverage:.1f}%")
        print(f"  Expected Coverage: {conf*100:.0f}%")
        print()
    
    # 5. Model Stability Analysis
    print("🔄 MODEL STABILITY ANALYSIS:")
    print("-" * 30)
    
    # Calculate metrics by prediction quantiles
    pred_quantiles = pd.qcut(y_pred, q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
    stability_analysis = pd.DataFrame({
        'Quantile': pred_quantiles,
        'Actual': y_test,
        'Predicted': y_pred,
        'Residual': residuals,
        'Abs_Error': abs_errors
    })
    
    stability_summary = stability_analysis.groupby('Quantile').agg({
        'Abs_Error': ['mean', 'std', 'median'],
        'Residual': ['mean', 'std']
    }).round(4)
    
    print("Performance by Prediction Quantiles:")
    print(stability_summary)
    
    return diagnostics_results, stability_summary

# Get predictions for the best model
best_predictions = model_predictions[best_model]
y_test_pred = best_predictions['test_pred']
y_test_actual = best_predictions['test_actual']

# Perform comprehensive diagnostics
X_train, X_test, y_train, y_test = data_splits
diagnostics, stability = comprehensive_model_diagnostics(residuals, X_test, y_test_actual, y_test_pred)

## 8. Feature Importance and Model Interpretability

In [ ]:
def analyze_feature_importance(X_train, y_train, feature_names):
    """
    Analyze feature importance using multiple methods
    """
    print("🎯 FEATURE IMPORTANCE ANALYSIS")
    print("=" * 40)
    
    # 1. Random Forest Feature Importance
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_importance = rf_model.feature_importances_
    
    # 2. Gradient Boosting Feature Importance
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)
    gb_importance = gb_model.feature_importances_
    
    # 3. Linear Regression Coefficients (absolute values)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train)
    lr_importance = np.abs(lr_model.coef_)
    lr_importance = lr_importance / lr_importance.sum()  # Normalize
    
    # Create feature importance DataFrame
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Random_Forest': rf_importance,
        'Gradient_Boosting': gb_importance,
        'Linear_Regression': lr_importance
    })
    
    # Calculate average importance
    importance_df['Average_Importance'] = importance_df[['Random_Forest', 'Gradient_Boosting', 'Linear_Regression']].mean(axis=1)
    importance_df = importance_df.sort_values('Average_Importance', ascending=False)
    
    print("\n📊 Feature Importance Rankings:")
    display(importance_df.round(4))
    
    # Visualize feature importance
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Random Forest Importance
    top_features_rf = importance_df.nlargest(10, 'Random_Forest')
    axes[0,0].barh(range(len(top_features_rf)), top_features_rf['Random_Forest'])
    axes[0,0].set_yticks(range(len(top_features_rf)))
    axes[0,0].set_yticklabels(top_features_rf['Feature'])
    axes[0,0].set_title('Random Forest Feature Importance')
    axes[0,0].grid(True, alpha=0.3)
    
    # Gradient Boosting Importance
    top_features_gb = importance_df.nlargest(10, 'Gradient_Boosting')
    axes[0,1].barh(range(len(top_features_gb)), top_features_gb['Gradient_Boosting'])
    axes[0,1].set_yticks(range(len(top_features_gb)))
    axes[0,1].set_yticklabels(top_features_gb['Feature'])
    axes[0,1].set_title('Gradient Boosting Feature Importance')
    axes[0,1].grid(True, alpha=0.3)
    
    # Linear Regression Coefficients
    top_features_lr = importance_df.nlargest(10, 'Linear_Regression')
    axes[1,0].barh(range(len(top_features_lr)), top_features_lr['Linear_Regression'])
    axes[1,0].set_yticks(range(len(top_features_lr)))
    axes[1,0].set_yticklabels(top_features_lr['Feature'])
    axes[1,0].set_title('Linear Regression Coefficient Importance')
    axes[1,0].grid(True, alpha=0.3)
    
    # Average Importance
    top_features_avg = importance_df.nlargest(10, 'Average_Importance')
    axes[1,1].barh(range(len(top_features_avg)), top_features_avg['Average_Importance'])
    axes[1,1].set_yticks(range(len(top_features_avg)))
    axes[1,1].set_yticklabels(top_features_avg['Feature'])
    axes[1,1].set_title('Average Feature Importance')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('🎯 Feature Importance Analysis', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
    return importance_df

# Analyze feature importance
feature_importance = analyze_feature_importance(data_splits[0], data_splits[2], feature_names)

## 9. Final Model Recommendations and Summary

In [ ]:
def generate_final_recommendations(model_results, diagnostics, feature_importance, best_model):
    """
    Generate final recommendations and summary
    """
    print("🎯 FINAL MODEL RECOMMENDATIONS & SUMMARY")
    print("=" * 50)
    
    # Best model summary
    best_model_row = model_results[model_results['Model'] == best_model].iloc[0]
    
    print(f"\n🏆 BEST MODEL: {best_model}")
    print("-" * 30)
    print(f"📊 Test R²: {best_model_row['Test_R2']:.4f}")
    print(f"📉 Test RMSE: {best_model_row['Test_RMSE']:.2f}")
    print(f"📈 Test MAE: {best_model_row['Test_MAE']:.2f}")
    print(f"🎯 Test MAPE: {best_model_row['Test_MAPE']:.2f}%")
    print(f"🔄 CV R² (Mean±Std): {best_model_row['CV_R2_Mean']:.4f}±{best_model_row['CV_R2_Std']:.4f}")
    print(f"⚠️  Overfitting Score: {best_model_row['Overfitting_Score']:.4f}")
    
    # Model ranking
    print("\n📈 MODEL RANKING (by Test R²):")
    print("-" * 35)
    ranking = model_results.sort_values('Test_R2', ascending=False)[['Model', 'Test_R2', 'Test_RMSE', 'Overfitting_Score']]
    for i, (_, row) in enumerate(ranking.iterrows(), 1):
        overfitting_status = "🔴 High" if row['Overfitting_Score'] > 0.1 else "🟡 Moderate" if row['Overfitting_Score'] > 0.05 else "🟢 Low"
        print(f"{i}. {row['Model']}: R²={row['Test_R2']:.4f}, RMSE={row['Test_RMSE']:.2f}, Overfitting={overfitting_status}")
    
    # Top features
    print("\n🎯 TOP 5 MOST IMPORTANT FEATURES:")
    print("-" * 40)
    top_features = feature_importance.head(5)
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        print(f"{i}. {row['Feature']}: {row['Average_Importance']:.4f}")
    
    # Model diagnostics summary
    print("\n🧪 MODEL DIAGNOSTICS SUMMARY:")
    print("-" * 35)
    
    if 'jb_p' in diagnostics:
        normality_status = "✅ Normal" if diagnostics['jb_p'] > 0.05 else "❌ Non-normal"
        print(f"Residual Normality (Jarque-Bera): {normality_status} (p={diagnostics['jb_p']:.4f})")
    
    if 'bp_p' in diagnostics:
        homoscedasticity_status = "✅ Homoscedastic" if diagnostics['bp_p'] > 0.05 else "❌ Heteroscedastic"
        print(f"Homoscedasticity (Breusch-Pagan): {homoscedasticity_status} (p={diagnostics['bp_p']:.4f})")
    
    # Recommendations
    print("\n💡 RECOMMENDATIONS:")
    print("-" * 20)
    
    # Performance-based recommendations
    if best_model_row['Test_R2'] > 0.9:
        print("✅ Excellent model performance (R² > 0.9)")
    elif best_model_row['Test_R2'] > 0.8:
        print("✅ Good model performance (R² > 0.8)")
    elif best_model_row['Test_R2'] > 0.7:
        print("⚠️  Moderate model performance (R² > 0.7) - consider feature engineering")
    else:
        print("❌ Poor model performance (R² < 0.7) - significant improvements needed")
    
    # Overfitting recommendations
    if best_model_row['Overfitting_Score'] > 0.1:
        print("🔴 High overfitting detected - consider regularization or more data")
    elif best_model_row['Overfitting_Score'] > 0.05:
        print("🟡 Moderate overfitting - monitor model generalization")
    else:
        print("🟢 Low overfitting - good generalization")
    
    # Feature recommendations
    if len(feature_importance) > 10:
        low_importance_features = feature_importance[feature_importance['Average_Importance'] < 0.01]
        if len(low_importance_features) > 0:
            print(f"🔧 Consider removing {len(low_importance_features)} low-importance features")
    
    # Model-specific recommendations
    if best_model in ['Random Forest', 'Gradient Boosting']:
        print("🌳 Tree-based model selected - good for non-linear relationships")
        print("💡 Consider hyperparameter tuning for further improvement")
    elif 'Regression' in best_model:
        print("📈 Linear model selected - interpretable but may miss non-linear patterns")
        print("💡 Consider polynomial features or interaction terms")
    
    # Data quality recommendations
    print("\n🔧 NEXT STEPS FOR IMPROVEMENT:")
    print("-" * 35)
    print("1. 📊 Collect more data if possible")
    print("2. 🔧 Engineer new features based on domain knowledge")
    print("3. 🎯 Perform hyperparameter tuning")
    print("4. 🔄 Try ensemble methods combining multiple models")
    print("5. 📈 Monitor model performance over time")
    print("6. 🧪 Validate model on new, unseen data")
    
    return {
        'best_model': best_model,
        'best_r2': best_model_row['Test_R2'],
        'best_rmse': best_model_row['Test_RMSE'],
        'overfitting_score': best_model_row['Overfitting_Score'],
        'top_features': top_features['Feature'].tolist()
    }

# Generate final recommendations
final_summary = generate_final_recommendations(model_results, diagnostics, feature_importance, best_model)

print("\n" + "="*60)
print("🎉 ANALYSIS COMPLETE!")
print("📊 This comprehensive EDA provides enhanced error analysis and model evaluation.")
print("💡 Use the insights above to improve your model further.")
print("="*60)

## 10. Export Results and Model Artifacts

In [ ]:
def export_analysis_results(model_results, feature_importance, diagnostics, final_summary):
    """
    Export analysis results to files
    """
    print("💾 EXPORTING ANALYSIS RESULTS")
    print("=" * 35)
    
    try:
        # Export model comparison results
        model_results.to_csv('model_comparison_results.csv', index=False)
        print("✅ Model comparison results exported to 'model_comparison_results.csv'")
        
        # Export feature importance
        feature_importance.to_csv('feature_importance_analysis.csv', index=False)
        print("✅ Feature importance analysis exported to 'feature_importance_analysis.csv'")
        
        # Export summary report
        with open('model_analysis_summary.txt', 'w') as f:
            f.write("EMPLOYEE COMPENSATION MODEL ANALYSIS SUMMARY\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Best Model: {final_summary['best_model']}\n")
            f.write(f"Test R²: {final_summary['best_r2']:.4f}\n")
            f.write(f"Test RMSE: {final_summary['best_rmse']:.2f}\n")
            f.write(f"Overfitting Score: {final_summary['overfitting_score']:.4f}\n\n")
            f.write("Top 5 Features:\n")
            for i, feature in enumerate(final_summary['top_features'], 1):
                f.write(f"{i}. {feature}\n")
        
        print("✅ Summary report exported to 'model_analysis_summary.txt'")
        
        print("\n📁 All results have been exported successfully!")
        
    except Exception as e:
        print(f"❌ Error exporting results: {e}")

# Export results
export_analysis_results(model_results, feature_importance, diagnostics, final_summary)